<a href="https://colab.research.google.com/github/bhyanx/Trabajo_Grupo_2/blob/main/GRUPO_2_MOVI_CITY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
pip install numpy pandas tensorflow scikit-learn matplotlib seaborn gradio

In [40]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr


# Paso 1: Cargar los datos

def cargar_datos():
    # Cargar el conjunto de datos
    datos = pd.read_csv("conductores_salud.csv")

    # Verificar columnas
    print("Columnas en el CSV:", datos.columns.tolist())

    # Convertir variables categóricas
    if 'Género' in datos.columns:
        datos['Género'] = datos['Género'].map({'Masculino': 1, 'Femenino': 0})
    else:
        raise KeyError("Falta la columna 'Género'.")

    if 'Consumo de Tabaco' in datos.columns:
        datos['Consumo de Tabaco'] = datos['Consumo de Tabaco'].map({'Sí': 1, 'No': 0})
    else:
        raise KeyError("Falta la columna 'Consumo de Tabaco'.")

    if 'Diagnóstico Médico Final' not in datos.columns:
        raise KeyError("Falta la columna 'Diagnóstico Médico Final'.")

    X = datos.drop('Diagnóstico Médico Final', axis=1)
    y = datos['Diagnóstico Médico Final']

    return X, y


# Paso 2: Preprocesar los datos

def preprocesar_datos(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test


# Paso 3: Crear modelo de red neuronal

def crear_modelo(input_shape):
    modelo = keras.Sequential([
        keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    modelo.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

    return modelo


# Paso 4: Entrenar el modelo

def entrenar_modelo(modelo, X_train, y_train):
    historia = modelo.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        verbose=0
    )
    return historia


# Paso 5: Evaluar el modelo

def evaluar_modelo(modelo, X_test, y_test):
    predicciones = modelo.predict(X_test)
    predicciones_binarias = (predicciones > 0.5).astype(int)

    print("Reporte de Clasificación:")
    print(classification_report(y_test, predicciones_binarias))

    cm = confusion_matrix(y_test, predicciones_binarias)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Matriz de Confusión')
    plt.ylabel('Etiquetas Reales')
    plt.xlabel('Predicciones')
    plt.show()


# Gráfico de precisión del modelo

def graficar_precision(historia):
    plt.figure(figsize=(10,5))
    plt.plot(historia.history['accuracy'], label='Precisión de Entrenamiento')
    plt.plot(historia.history['val_accuracy'], label='Precisión de Validación')
    plt.title('Precisión del Modelo')
    plt.xlabel('Época')
    plt.ylabel('Precisión')
    plt.legend()
    plt.grid(True)
    plt.show()


# Interfaz con Gradio

def predecir_diagnostico(edad, genero, imc, colesterol, trigliceridos,
                        frecuencia_cardiaca, presion_sistolica, presion_diastolica,
                        nivel_estres, consumo_tabaco, actividad_fisica):

    try:
        # Convertir género a numérico
        genero_num = 1 if genero == "Masculino" else 0
        tabaco_num = 1 if consumo_tabaco == "Sí" else 0

        entrada = [[edad, genero_num, imc, colesterol, trigliceridos,
                    frecuencia_cardiaca, presion_sistolica, presion_diastolica,
                    nivel_estres, tabaco_num, actividad_fisica]]

        entrada_scaled = scaler.transform(entrada)
        probabilidad = modelo.predict(entrada_scaled)[0][0]

        clase = 1 if probabilidad > 0.5 else 0

        return {
            "Alto Riesgo": float(probabilidad),
            "Bajo Riesgo": float(1 - probabilidad)
        }

    except Exception as e:
        print("Error durante la predicción:", str(e))
        return {"Error": "Ocurrió un problema al procesar los datos."}


# Función principal

def main():
    global modelo, scaler  # Hacer accesibles para Gradio

    print("Cargando datos...")
    X, y = cargar_datos()

    print("Preprocesando datos...")
    X_train, X_test, y_train, y_test = preprocesar_datos(X, y)

    print("Entrenando modelo...")
    modelo = crear_modelo(X_train.shape[1])
    historia = entrenar_modelo(modelo, X_train, y_train)

    print("Evaluando modelo...")
    evaluar_modelo(modelo, X_test, y_test)

    print("Mostrando gráficos...")
    graficar_precision(historia)

    print("Guardando escalador...")
    scaler = StandardScaler()
    scaler.fit(X)  # Escalador final ajustado a todos los datos

    print("Iniciando interfaz Gradio...")
    interfaz = gr.Interface(fn=predecir_diagnostico,
                            inputs=[
                                gr.Slider(minimum=18, maximum=100, label="Edad"),
                                gr.Radio(choices=["Femenino", "Masculino"], label="Género"),
                                gr.Number(label="IMC"),
                                gr.Number(label="Colesterol (mg/dL)"),
                                gr.Number(label="Triglicéridos (mg/dL)"),
                                gr.Number(label="Frecuencia Cardíaca (ppm)"),
                                gr.Number(label="Presión Sistólica"),
                                gr.Number(label="Presión Diastólica"),
                                gr.Slider(minimum=1, maximum=10, label="Nivel de Estrés (1-10)"),
                                gr.Radio(choices=["Sí", "No"], label="Consumo de Tabaco"),
                                gr.Number(label="Actividad Física (horas/semana)")
                            ],
                            outputs="label",
                            title="Modelo Predictivo - Diagnóstico Médico Final",
                            description="Ingrese los datos del conductor para predecir su diagnóstico médico final.")
    interfaz.launch()

## EJECUCIÓN DEL PROGRAMA
if __name__ == '__main__':
    main()

Cargando datos...
Columnas en el CSV: ['Edad', 'Género', 'IMC', 'Colesterol (mg/dL)', 'Triglicéridos (mg/dL)', 'Frecuencia Cardíaca (ppm)', 'Presión Sistólica', 'Presión Diastólica', 'Nivel de Estrés (1-10)', 'Consumo de Tabaco', 'Actividad Física (horas/semana)', 'Diagnóstico Médico Final']
Preprocesando datos...
Entrenando modelo...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Evaluando modelo...
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.86      0.78      0.82        54
           1       0.92      0.95      0.94       146

    accuracy                           0.91       200
   macro avg       0.89      0.86      0.88       200
weighted avg       0.90      0.91      0.90       200

Mostrando gráficos...
Guardando escalador...
Iniciando interfaz Gradio...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7cf97d307eaabb11b3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the workin